<a href="https://colab.research.google.com/github/MichelePresti/NAS_MachineLearningDeepLearning/blob/main/NaswotREA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Define Config


In [42]:
config = {}
max_uid = 15625
#@title ##Configuration Info { run: "auto" }
#configuration by param
dataset = "ImageNet16" #@param {type:"string"} ["cifar10", "cifar100", "ImageNet16"]
run_id =  1# @param {type:"integer"}
trial =  2#@param {type:"integer"}
n_random =  10#@param {type:"integer"}
point = '2a' # @param ['2a', '2b']
imagenet_path = 'Use only if dataset=Imagenet16' #@param{type:"string"}
use_default_path = True #@param{type:"boolean"}
n_evolution = 2#@param{type: "integer"}
n_arch_distance = 2#@param{type: "integer"}
n_survivor = 1#@param{type:"integer"}
population_size = 10#@param{type:"integer"}
proxy_type = "ReLU" #@param {type: "string"} ["ReLU", "SynFlow"]


config['score'] = 'hook_logdet'
config['nasspace'] = 'nasbench201'
config['augtype'] = 'none'
config['dataset'] = dataset
config['maxofn'] = 3
config['batch_size'] = 128
config['seed'] = 1
config['run_id'] = run_id
config['dataset_id'] = 'CIFAR10'
config['start_uid'] = 0 
config['stop_uid'] =  15000 
config['trial'] = trial
config['n_random'] = n_random
config['point'] = point
config['imagenet_path'] = '/content/drive/MyDrive/ImageNet16' if use_default_path else imagenet_path
config['n_evolution'] = n_evolution
config['n_arch_distance'] = n_arch_distance
config['n_survivor'] = n_survivor
config['population_size'] = population_size
config['proxy_type'] = proxy_type

#max 15625 stop_uid

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Import NAS Bench API

In [3]:
%%capture
!git clone https://github.com/MichelePresti/NAS_MachineLearningDeepLearning

In [4]:
!cp -r /content/NAS_MachineLearningDeepLearning/neural_model .

In [5]:
import pandas as pd

def get_arch_config_by_dataset(dataset) -> pd.DataFrame:
    """
    This function return the architectures config by dataset in a pandas dataframe.
    PARAMETERS:
       dataset= string among [cifar10, cifar100, imaginet]
    """
    if(dataset == 'cifar10'):
        df = pd.read_csv('/content/NAS_MachineLearningDeepLearning/nas_bench_201__CIFAR10_config.csv', header=0)
        return df
    if(dataset == 'cifar100'):
      df = pd.read_csv('/content/NAS_MachineLearningDeepLearning/nas_bench_201__CIFAR100_config.csv', header=0)
      return df
    if(dataset == 'ImageNet16'):
      df = pd.read_csv('/content/NAS_MachineLearningDeepLearning/nas_bench_201__ImageNet16_config.csv', header=0)
      return df
    else: 
      print('Dataset name not valid')
      return None

def get_standard_config(csv_config: pd.DataFrame) -> dict:
    res = {}
    res['name'] = csv_config.iloc[0]['name']
    res['C'] = csv_config.iloc[0]['C']
    res['N'] = csv_config.iloc[0]['N']
    res['arch_str'] = csv_config.iloc[0]['arch_str']
    res['num_classes'] = 1
    return res
        

In [6]:
import pandas as pd

searchspace = get_arch_config_by_dataset(config['dataset'])
searchspace
searchspace.to_csv('searchspace_daprocessare.csv')

#Import Dataset

In [16]:
##################################################
# Copyright (c) Xuanyi Dong [GitHub D-X-Y], 2019 #
##################################################
import os, sys, hashlib, torch
import numpy as np
from PIL import Image
import torch.utils.data as data

if sys.version_info[0] == 2:
    import cPickle as pickle
else:
    import pickle


def calculate_md5(fpath, chunk_size=1024 * 1024):
    md5 = hashlib.md5()
    with open(fpath, "rb") as f:
        for chunk in iter(lambda: f.read(chunk_size), b""):
            md5.update(chunk)
    return md5.hexdigest()


def check_md5(fpath, md5, **kwargs):
    return md5 == calculate_md5(fpath, **kwargs)


def check_integrity(fpath, md5=None):
    if not os.path.isfile(fpath):
        return False
    if md5 is None:
        return True
    else:
        return check_md5(fpath, md5)


class ImageNet16(data.Dataset):
    # http://image-net.org/download-images
    # A Downsampled Variant of ImageNet as an Alternative to the CIFAR datasets
    # https://arxiv.org/pdf/1707.08819.pdf

    train_list = [
        ["train_data_batch_1", "27846dcaa50de8e21a7d1a35f30f0e91"],
        ["train_data_batch_2", "c7254a054e0e795c69120a5727050e3f"],
        ["train_data_batch_3", "4333d3df2e5ffb114b05d2ffc19b1e87"],
        ["train_data_batch_4", "1620cdf193304f4a92677b695d70d10f"],
        ["train_data_batch_5", "348b3c2fdbb3940c4e9e834affd3b18d"],
        ["train_data_batch_6", "6e765307c242a1b3d7d5ef9139b48945"],
        ["train_data_batch_7", "564926d8cbf8fc4818ba23d2faac7564"],
        ["train_data_batch_8", "f4755871f718ccb653440b9dd0ebac66"],
        ["train_data_batch_9", "bb6dd660c38c58552125b1a92f86b5d4"],
        ["train_data_batch_10", "8f03f34ac4b42271a294f91bf480f29b"],
    ]
    valid_list = [
        ["val_data", "3410e3017fdaefba8d5073aaa65e4bd6"],
    ]

    def __init__(self, root, train, transform, use_num_of_class_only=None):
        self.root = root
        self.transform = transform
        self.train = train  # training set or valid set
        if not self._check_integrity():
            raise RuntimeError("Dataset not found or corrupted.")

        if self.train:
            downloaded_list = self.train_list
        else:
            downloaded_list = self.valid_list
        self.data = []
        self.targets = []

        # now load the picked numpy arrays
        for i, (file_name, checksum) in enumerate(downloaded_list):
            file_path = os.path.join(self.root, file_name)
            # print ('Load {:}/{:02d}-th : {:}'.format(i, len(downloaded_list), file_path))
            with open(file_path, "rb") as f:
                if sys.version_info[0] == 2:
                    entry = pickle.load(f)
                else:
                    entry = pickle.load(f, encoding="latin1")
                self.data.append(entry["data"])
                self.targets.extend(entry["labels"])
        self.data = np.vstack(self.data).reshape(-1, 3, 16, 16)
        self.data = self.data.transpose((0, 2, 3, 1))  # convert to HWC
        if use_num_of_class_only is not None:
            assert (
                isinstance(use_num_of_class_only, int)
                and use_num_of_class_only > 0
                and use_num_of_class_only < 1000
            ), "invalid use_num_of_class_only : {:}".format(use_num_of_class_only)
            new_data, new_targets = [], []
            for I, L in zip(self.data, self.targets):
                if 1 <= L <= use_num_of_class_only:
                    new_data.append(I)
                    new_targets.append(L)
            self.data = new_data
            self.targets = new_targets

    def __repr__(self):
        return "{name}({num} images, {classes} classes)".format(
            name=self.__class__.__name__,
            num=len(self.data),
            classes=len(set(self.targets)),
        )

    def __getitem__(self, index):
        img, target = self.data[index], self.targets[index] - 1

        img = Image.fromarray(img)

        if self.transform is not None:
            img = self.transform(img)

        return img, target

    def __len__(self):
        return len(self.data)

    def _check_integrity(self):
        root = self.root
        for fentry in self.train_list + self.valid_list:
            filename, md5 = fentry[0], fentry[1]
            fpath = os.path.join(root, filename)
            if not check_integrity(fpath, md5):
                return False
        return True



In [17]:
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision
import torch

def get_dataset(dataset) -> DataLoader:
    """
    This function return the dataset given its name in torch DataLoader format.
    PARAMETERS:
       dataset= string among [cifar10, cifar100, imaginet]
    """

    if dataset == 'cifar10':
        mean = [x / 255 for x in [125.3, 123.0, 113.9]]
        std = [x / 255 for x in [63.0, 62.1, 66.7]]
        lists = [transforms.RandomHorizontalFlip(), transforms.RandomCrop(32, padding=4), transforms.ToTensor(),
              transforms.Normalize(mean, std)]
        transform = transforms.Compose(lists)
        trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                                download=True, transform=transform)
        train_dt = torch.utils.data.DataLoader(trainset, batch_size=config['batch_size'],
                                                  shuffle=True, num_workers=2)
    elif dataset == 'cifar100':
        mean = [x / 255 for x in [129.3, 124.1, 112.4]]
        std = [x / 255 for x in [68.2, 65.4, 70.4]]
        lists = [transforms.RandomHorizontalFlip(), transforms.RandomCrop(32, padding=4), transforms.ToTensor(),
              transforms.Normalize(mean, std)]
        transform = transforms.Compose(lists)
        trainset = torchvision.datasets.CIFAR100(root='./data', train=True,
                                                download=True, transform=transform)
        train_dt = torch.utils.data.DataLoader(trainset, batch_size=config['batch_size'],
                                                  shuffle=True, num_workers=2)
    elif dataset.startswith('ImageNet16'):
        mean = [x / 255 for x in [122.68, 116.66, 104.01]]
        std = [x / 255 for x in [63.22, 61.26, 65.09]]
        lists = [transforms.RandomHorizontalFlip(), transforms.RandomCrop(16, padding=2), transforms.ToTensor(),
                 transforms.Normalize(mean, std)]
        transform = transforms.Compose(lists)
        trainset = ImageNet16(config['imagenet_path'], True, transform, 120)
        train_dt = torch.utils.data.DataLoader(trainset, batch_size=config['batch_size'],
                                                  shuffle=True, num_workers=2)
    else:
        raise TypeError("Unknow dataset : {:}".format(dataset))

    return train_dt

In [18]:
train_dt = get_dataset(config['dataset'])

#Define Score

In [19]:
import numpy as np

def get_batch_jacobian(net, x, target, device, args=None):
    net.zero_grad()
    x.requires_grad_(True)
    y, out = net(x)
    y.backward(torch.ones_like(y))
    jacob = x.grad.detach()
    return jacob, target.detach(), y.detach(), out.detach()


def hooklogdet(K, labels=None):
    s, ld = np.linalg.slogdet(K)
    return ld


def score_network(network, x, x2, target, device):
    jacobs, labels, y, out = get_batch_jacobian(network, x, target, device, config)
    network(x2.to(device))
    value = hooklogdet(network.K, target)
    return value

#Search Algorithm

In [20]:
import time
import random
from neural_model.neural_model import get_cell_net
from tqdm.notebook import tqdm
from tqdm.contrib.telegram import tqdm, trange

def naswot_search_n2(run_id, dataset_id, device, n):
    result = {'uid': [], 'score': [], 'elapsed_time': []}
    n = config['n_random']
    trial = config['trial']
    point = config['point']
    l = random.sample(range(max_uid), n)
    # for uid in tqdm(l,\
    #    token='5374697833:AAFXE6wumxuSFFwpSzSmEQ6WL4jl8OBVOqw',\
    #    chat_id='142397010',\
    #    desc=f'result_run{run_id}_dataset{dataset_id}_point_{point}'):
    for uid in l:
      #i = uid-start_uid
      net_config: pd.DataFrame = searchspace.loc[searchspace['uid'] == uid]
      net_config: dict = get_standard_config(net_config)
      network = get_cell_net(net_config)
      try:
          start = time.time()
          if 'hook_' in config['score']:

              def counting_forward_hook(module_hook, inp, out):
                  try:
                      if hasattr(module_hook, 'visited_backwards') and not module_hook.visited_backwards:
                          return
                      if isinstance(inp, tuple):
                          inp = inp[0]
                      inp = inp.view(inp.size(0), -1)
                      x = (inp > 0).float()
                      K = x @ x.t()
                      K2 = (1. - x) @ (1. - x.t())
                      if hasattr(network, 'K'):
                        network.K = network.K + K.cpu().numpy() + K2.cpu().numpy()
                      else: 
                        network.K = K.cpu().numpy() + K2.cpu().numpy()
                  except Exception as exception:
                      print(exception)
                      pass


              def counting_backward_hook(module_hook, inp, out):
                  module_hook.visited_backwards = True

              j = []
              for name, module in network.named_modules():
                  j.append(name)
                  if 'ReLU' in str(type(module)):
                      module.register_forward_hook(counting_forward_hook)
                      module.register_backward_hook(counting_backward_hook)

          #print('NUM MODULES IN ARCHITECTURES', len(j))
          # Starting score algorithm
          network = network.to(device)
          ## start time
          #start = time.time()
          random.seed(config['seed'])
          np.random.seed(config['seed'])
          torch.manual_seed(config['seed'])
          s = []
          for j in range(config['maxofn']):
              data_iterator = iter(train_dt)
              x, target = next(data_iterator)
              x2 = torch.clone(x)
              x2 = x2.to(device)
              x, target = x.to(device), target.to(device)
              jacobs, labels, y, out = get_batch_jacobian(network, x, target, device, config)
              if 'hook_' in config['score']:
                  network(x2.to(device))
                  value = hooklogdet(network.K, target)
                  s.append(value)
              else:
                  value = hooklogdet(network.K, target)
                  s.append(value)
          #print(f'Score (uid {uid}): {np.mean(s)}')
          stop = time.time()
          result['uid'].append(uid)
          result['score'].append(np.mean(s))
          result['elapsed_time'].append(stop-start)
          #print(f'Elapsed time (uid {uid}): {stop-start}')
      except Exception as e:
          print(e)
    df = pd.DataFrame.from_dict(result)
    result = {'uid': [], 'score': [], 'elapsed_time': []}
    df.to_csv(f'./result_run{run_id}_{trial}_{dataset_id}.csv')
    
    return 

#2a or 2b

In [21]:
import pandas as pd

df = pd.read_csv('/content/NAS_MachineLearningDeepLearning/Cifar10Result.csv')
try: 
  df.drop(columns=['Unnamed: 0'], inplace=True)
except:
  print("Already dropped")

acc_df = df


In [22]:
import time
import random
from neural_model.neural_model import get_cell_net
from tqdm.notebook import tqdm
from tqdm.contrib.telegram import tqdm, trange


def naswot_search_n(run_id, dataset, device, n, trial):
    result = {'uid': [], 'score': [], 'elapsed_time': []}
    n = config['n_random']
    #trial = config['trial']
    #random.seed(i)
    l = random.sample(range(max_uid), n)
    best_score, best_acc, best_net = 0, 0, 0
    best = {}
    best['score'] = 0
    best['acc'] = 0
    best['uid'] = 0
    print(l)
    # for uid in tqdm(l,\
    #    token='5374697833:AAFXE6wumxuSFFwpSzSmEQ6WL4jl8OBVOqw',\
    #    chat_id='142397010',\
    #    desc=f'result_run{run_id}_{point}_dataset{dataset_id}_'):
    for uid in l:
      #i = uid-start_uid
      net_config: pd.DataFrame = searchspace.loc[searchspace['uid'] == uid]
      net_config: dict = get_standard_config(net_config)
      network = get_cell_net(net_config)
      try:
          start = time.time()
          if 'hook_' in config['score']:

              def counting_forward_hook(module_hook, inp, out):
                  try:
                      if hasattr(module_hook, 'visited_backwards') and not module_hook.visited_backwards:
                          return
                      if isinstance(inp, tuple):
                          inp = inp[0]
                      inp = inp.view(inp.size(0), -1)
                      x = (inp > 0).float()
                      K = x @ x.t()
                      K2 = (1. - x) @ (1. - x.t())
                      if hasattr(network, 'K'):
                        network.K = network.K + K.cpu().numpy() + K2.cpu().numpy()
                      else: 
                        network.K = K.cpu().numpy() + K2.cpu().numpy()
                  except Exception as exception:
                      print(exception)
                      pass


              def counting_backward_hook(module_hook, inp, out):
                  module_hook.visited_backwards = True

              j = []
              for name, module in network.named_modules():
                  j.append(name)
                  if 'ReLU' in str(type(module)):
                      module.register_forward_hook(counting_forward_hook)
                      module.register_backward_hook(counting_backward_hook)

          #print('NUM MODULES IN ARCHITECTURES', len(j))
          # Starting score algorithm
          
          network = network.to(device)
          ## start time
          #start = time.time()
          #random.seed(config['seed'])
          #np.random.seed(config['seed'])
          #torch.manual_seed(config['seed'])
          s = []
          for j in range(config['maxofn']):
              data_iterator = iter(train_dt)
              x, target = next(data_iterator)
              x2 = torch.clone(x)
              x2 = x2.to(device)
              x, target = x.to(device), target.to(device)
              jacobs, labels, y, out = get_batch_jacobian(network, x, target, device, config)
              if 'hook_' in config['score']:
                  network(x2.to(device))
                  value = hooklogdet(network.K, target)
                  s.append(value)
              else:
                  value = hooklogdet(network.K, target)
                  s.append(value)
          acc = acc_df['valid-accuracy'].iloc[uid]
          print(f'Score (uid {uid}): {np.mean(s)}, Accuracy: {acc}')
          stop = time.time()
          result['uid'].append(uid)
          result['score'].append(np.mean(s))
          score = np.mean(s)
          result['elapsed_time'].append(stop-start)

          if config['point'] == '2a':
          #point 2a
            if score > best_score:
              best_score = score
              best_net = uid
              best_acc = acc
          else:
            #point 2b
            
            if acc > best_acc:
              best_acc = acc
              best_score = score
              best_net = uid

          #print(f'Elapsed time (uid {uid}): {stop-start}')
      except Exception as e:
          print(e)
    df = pd.DataFrame.from_dict(result)
    result = {'uid': [], 'score': [], 'elapsed_time': []}
    df.to_csv(f'./result_run{run_id}_trial{trial}_{point}_{dataset}.csv')
    print(f'Best Network: {best_net}; Score: {best_score}; Accuracy: {best_acc}')
    best = {}
    best['acc'] = best_acc
    best['score'] = best_score
    best['uid'] = best_net
    series_net = pd.Series(best)
    series_net.to_csv(f'best_network_{run_id}_trial{trial}_{dataset}.csv')
    return 

#All Search

In [23]:
import time
import random
from neural_model.neural_model import get_cell_net
from tqdm.notebook import tqdm
from tqdm.contrib.telegram import tqdm, trange

def search_all(run_id, dataset_id, device, start_uid, stop_uid):
    result = {'uid': [], 'score': [], 'elapsed_time': []}
    #l = random.sample(range(max_uid), 100)
    #for uid in tqdm(range(start_uid, stop_uid)):
    for uid in tqdm(range(start_uid, stop_uid),\
       token='5374697833:AAFXE6wumxuSFFwpSzSmEQ6WL4jl8OBVOqw',\
       chat_id='142397010',\
       desc=f'result_run{run_id}_{start_uid//1000}_dataset{dataset_id}_'):
      i = uid-start_uid
      net_config: pd.DataFrame = searchspace.loc[searchspace['uid'] == uid]
      net_config: dict = get_standard_config(net_config)
      network = get_cell_net(net_config)
      try:
          start = time.time()
          if 'hook_' in config['score']:

              def counting_forward_hook(module_hook, inp, out):
                  try:
                      if hasattr(module_hook, 'visited_backwards') and not module_hook.visited_backwards:
                          return
                      if isinstance(inp, tuple):
                          inp = inp[0]
                      inp = inp.view(inp.size(0), -1)
                      x = (inp > 0).float()
                      K = x @ x.t()
                      K2 = (1. - x) @ (1. - x.t())
                      if hasattr(network, 'K'):
                        network.K = network.K + K.cpu().numpy() + K2.cpu().numpy()
                      else: 
                        network.K = K.cpu().numpy() + K2.cpu().numpy()
                  except Exception as exception:
                      print(exception)
                      pass


              def counting_backward_hook(module_hook, inp, out):
                  module_hook.visited_backwards = True

              j = []
              for name, module in network.named_modules():
                  j.append(name)
                  if 'ReLU' in str(type(module)):
                      module.register_forward_hook(counting_forward_hook)
                      module.register_backward_hook(counting_backward_hook)

          #print('NUM MODULES IN ARCHITECTURES', len(j))
          # Starting score algorithm
          network = network.to(device)
          ## start time
          #start = time.time()
          random.seed(config['seed'])
          np.random.seed(config['seed'])
          torch.manual_seed(config['seed'])
          s = []
          for j in range(config['maxofn']):
              data_iterator = iter(train_dt)
              x, target = next(data_iterator)
              x2 = torch.clone(x)
              x2 = x2.to(device)
              x, target = x.to(device), target.to(device)
              jacobs, labels, y, out = get_batch_jacobian(network, x, target, device, config)
              if 'hook_' in config['score']:
                  network(x2.to(device))
                  value = hooklogdet(network.K, target)
                  s.append(value)
              else:
                  value = hooklogdet(network.K, target)
                  s.append(value)
          #print(f'Score (uid {uid}): {np.mean(s)}')
          stop = time.time()
          result['uid'].append(uid)
          result['score'].append(np.mean(s))
          result['elapsed_time'].append(stop-start)
          #print(f'Elapsed time (uid {uid}): {stop-start}')
          if i + 1 % 1000 == 0:
            df = pd.DataFrame.from_dict(result)
            result = {'uid': [], 'score': [], 'elapsed_time': []}
            df.to_csv(f'./NASWOT_result_run{run_id}_dataset{dataset_id}_{i}_.csv')
      except Exception as e:
          print(e)
    df = pd.DataFrame.from_dict(result)
    result = {'uid': [], 'score': [], 'elapsed_time': []}
    df.to_csv(f'./result_run{run_id}_{start_uid//1000}_dataset{dataset_id}_LastRecords_.csv')
    
    return 

#Save Result

In [15]:
#path = f'./result_run{run_id}_{start_uid//1000}_dataset{dataset_id}_LastRecords_.csv'
path = f'./result_run{run_id}_{trial}_{dataset_id}.csv'
path_net = f'best_network_{run_id}_trial{trial}_{dataset_id}.csv'

NameError: ignored

In [ ]:
!cp -r $path /content/drive/MyDrive/project
!cp -r $path_net /content/drive/MyDrive/project

In [ ]:
# %%capture
# import requests

# TOKEN = '5374697833:AAFXE6wumxuSFFwpSzSmEQ6WL4jl8OBVOqw'
# CHAT_ID = '142397010'
# SEND_URL = f'https://api.telegram.org/bot{TOKEN}/sendMessage'
# your_message = f"RUN COMPLETED!\nInformation:\n\t\t\t\t\t\t\t\tRun_Id:\t{run_id}\n\t\t\t\t\t\t\t\tTrial:{trial}\n\t\t\t\t\t\t\t\tDataset_Id:\t{dataset_id}"
# requests.post(SEND_URL, json={'chat_id': CHAT_ID, 'text': your_message}) 

# SynFlow Proxy Evolution Algorithm ⚛ 👾



In [29]:
!cp -r /content/NAS_MachineLearningDeepLearning/ZeroCostNas .

In [30]:
import time
import random
import pandas as pd
from neural_model.neural_model import get_cell_net

"""
Synflow proxy implementation
TO BE IMPLEMENTED
"""
from ZeroCostNas.foresight.models import *
from ZeroCostNas.foresight.pruners import *
from ZeroCostNas.foresight.dataset import *
from ZeroCostNas.foresight.weight_initializers import init_net


def synflow_search(dataset, device, population) -> pd.DataFrame:
    result = {'uid': [], 'score': [], 'elapsed_time': [], 'accuracy': []}
    for uid in population:
      net_config: pd.DataFrame = searchspace.loc[searchspace['uid'] == uid]
      net_config: dict = get_standard_config(net_config)
      network = get_cell_net(net_config)
      try:
          start = time.time()
          if 'hook_' in config['score']:

              def counting_forward_hook(module_hook, inp, out):
                  try:
                      if hasattr(module_hook, 'visited_backwards') and not module_hook.visited_backwards:
                          return
                      if isinstance(inp, tuple):
                          inp = inp[0]
                      inp = inp.view(inp.size(0), -1)
                      x = (inp > 0).float()
                      K = x @ x.t()
                      K2 = (1. - x) @ (1. - x.t())
                      if hasattr(network, 'K'):
                        network.K = network.K + K.cpu().numpy() + K2.cpu().numpy()
                      else: 
                        network.K = K.cpu().numpy() + K2.cpu().numpy()
                  except Exception as exception:
                      print(exception)
                      pass


              def counting_backward_hook(module_hook, inp, out):
                  module_hook.visited_backwards = True

              j = []
              for name, module in network.named_modules():
                  j.append(name)
                  if 'ReLU' in str(type(module)):
                      module.register_forward_hook(counting_forward_hook)
                      module.register_backward_hook(counting_backward_hook)
          network = network.to(device)
          s = []
          for j in range(config['maxofn']):
              data_iterator = iter(train_dt)
              x, target = next(data_iterator)
              x2 = torch.clone(x)
              x2 = x2.to(device)
              x, target = x.to(device), target.to(device)
              jacobs, labels, y, out = get_batch_jacobian(network, x, target, device, config)
              if 'hook_' in config['score']:
                  network(x2.to(device))
                  value = hooklogdet(network.K, target)
                  s.append(value)
              else:
                  value = hooklogdet(network.K, target)
                  s.append(value)
          acc = acc_df['valid-accuracy'].iloc[uid]
          print(f'Score (uid {uid}): {np.mean(s)}, Accuracy: {acc}')
          stop = time.time()
          result['uid'].append(uid)
          result['score'].append(np.mean(s))
          result['accuracy'].append(acc)
          score = np.mean(s)
          result['elapsed_time'].append(stop-start)
      except Exception as e:
          print(e)
    df = pd.DataFrame.from_dict(result)

    result = {'uid': [], 'score': [], 'elapsed_time': []}
    return df

# Regularized Evolution Algorithm NASWOT 🥇





In [28]:
import time
import random
import pandas as pd
from neural_model.neural_model import get_cell_net

"""
NAS WOT Algorithm
"""

def naswot_search(dataset, device, population) -> pd.DataFrame:
    result = {'uid': [], 'score': [], 'elapsed_time': [], 'accuracy': []}
    for uid in population:
      net_config: pd.DataFrame = searchspace.loc[searchspace['uid'] == uid]
      net_config: dict = get_standard_config(net_config)
      network = get_cell_net(net_config)
      try:
          start = time.time()
          if 'hook_' in config['score']:

              def counting_forward_hook(module_hook, inp, out):
                  try:
                      if hasattr(module_hook, 'visited_backwards') and not module_hook.visited_backwards:
                          return
                      if isinstance(inp, tuple):
                          inp = inp[0]
                      inp = inp.view(inp.size(0), -1)
                      x = (inp > 0).float()
                      K = x @ x.t()
                      K2 = (1. - x) @ (1. - x.t())
                      if hasattr(network, 'K'):
                        network.K = network.K + K.cpu().numpy() + K2.cpu().numpy()
                      else: 
                        network.K = K.cpu().numpy() + K2.cpu().numpy()
                  except Exception as exception:
                      print(exception)
                      pass


              def counting_backward_hook(module_hook, inp, out):
                  module_hook.visited_backwards = True

              j = []
              for name, module in network.named_modules():
                  j.append(name)
                  if 'ReLU' in str(type(module)):
                      module.register_forward_hook(counting_forward_hook)
                      module.register_backward_hook(counting_backward_hook)
          network = network.to(device)
          s = []
          for j in range(config['maxofn']):
              data_iterator = iter(train_dt)
              x, target = next(data_iterator)
              x2 = torch.clone(x)
              x2 = x2.to(device)
              x, target = x.to(device), target.to(device)
              jacobs, labels, y, out = get_batch_jacobian(network, x, target, device, config)
              if 'hook_' in config['score']:
                  network(x2.to(device))
                  value = hooklogdet(network.K, target)
                  s.append(value)
              else:
                  value = hooklogdet(network.K, target)
                  s.append(value)
          acc = acc_df['valid-accuracy'].iloc[uid]
          print(f'Score (uid {uid}): {np.mean(s)}, Accuracy: {acc}')
          stop = time.time()
          result['uid'].append(uid)
          result['score'].append(np.mean(s))
          result['accuracy'].append(acc)
          score = np.mean(s)
          result['elapsed_time'].append(stop-start)
      except Exception as e:
          print(e)
    df = pd.DataFrame.from_dict(result)

    result = {'uid': [], 'score': [], 'elapsed_time': []}
    return df

# Random Search Algorithm 🎳

---

### Algorithm Steps

1. Sample N Random Architectures
2. Run Scoring Algorithm On Them
3. Choose and store the best performing
4. Repeat N Times and then take the best one of the BestArchitecturesArray

In [ ]:
import time
import requests
import random
import os
import pandas as pd
from tabulate import tabulate
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
os.environ['WANDB_CONSOLE'] = 'off'
start = time.time()

run_id = config['run_id']
dataset = config['dataset']
start_uid = config['start_uid']
stop_uid = config['stop_uid']
n = config['n_random']    # N size of random sample
trial = config['trial']
proxy_type = config['proxy_type']
survivors = {}

print('*******************************')
print('Running Random Search algorithm')
print('Parameters:')
print(f'Dataset: {dataset}')
print(f'Num Round: {trial}')
print(f'Scoring Algorithm: {proxy_type}')
print('*******************************')


for i in range(trial):
  print(f"Round {i}")
  population = random.sample(range(max_uid), config['n_random'])
  if config['proxy_type'] == 'ReLU':
    trained_population = naswot_search(dataset, device, population)
  else: 
    trained_population = synflow_search(dataset, device, population)
  trained_population.sort_values(by=['score'], ascending=False, inplace=True)
  if len(survivors) > 0:
    survivors: pd.DataFrame = survivors.append(trained_population.head(config['n_survivor']), ignore_index=True)
  else:
    survivors = trained_population.head(config['n_survivor'])

survivors.sort_values(by=['score'], ascending=False, inplace=True)
survivors = survivors.head(1)

survivors.to_csv('RandomSearch.csv')
stop = time.time()

total_time = stop - start
print('*****************************************************************')
print(f'Best performing net with RandomSearch')
print(tabulate(survivors, headers='keys', tablefmt='psql', showindex=False))
print(f'Total time for search over all searchspace: {total_time}')
print('*****************************************************************')


# Run Evolution Algorithm ✨
---



### Algorithm Steps


1.   Get N Random Architectures Called Population
2.   Run Scoring Algorithm On Population
3.   Take N Survivor, Choose As The Best Score
4.   Create New Generation With Architecture At N Distance From The Survivor
5.   Repeat From 2 For N Evolution Era

In [39]:
"""
Architecture Configuration Class
"""

class Architecture:
    def __init__(self, arch):
        self.architecture = arch
        self.graph = {'0': [], '1':[], '2': []}
        for token in arch.split('+'):
            tmp = list(filter(lambda x: x != '', token.split('|')))
            for x in tmp:
                op = x.split('~')
                self.graph[str(op[1])].append(op[0])
            
    def distance(self, obj):
        differences = 0
        if isinstance(obj, Architecture):
            for key in self.graph.keys():
                differences += len([i for i, j in zip(self.graph[key], obj.graph[key]) if i != j])
            # print('DIFF:' + str(differences))
            return differences
        else:
            print('Invalid Object')
            return -1
            
    def print_architecture(self):
        print(self.architecture)

In [40]:
import time
import random

"""
Find all architectures with a distance n (max), the distance is calculated as Hamming Distance
"""

def find_arch_n_dist(survivors, max_dist, anchestors):
  population = []
  for uid in survivors['uid']:
    net_config: pd.DataFrame = searchspace.loc[searchspace['uid'] == uid]
    net_config: dict = get_standard_config(net_config)
    net_config = Architecture(net_config['arch_str'])
    for id in range(0, max_uid):
      candidate: pd.DataFrame = searchspace.loc[searchspace['uid'] == id]
      candidate: dict = get_standard_config(candidate)
      candidate = Architecture(candidate['arch_str'])
      dist = net_config.distance(candidate)
      # print('Distance: ' + str(dist))
      # if id not in anchestors and dist <= max_dist:
      if dist <= max_dist:
        anchestors.append(id)
        population.append(id)

  population = random.sample(population, config['population_size']) if len(population) > config['population_size'] else []
  population.append(*survivors['uid'])
  return population




In [ ]:
# Get N Random Samples

import random
import pandas as pd

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
dataset = config['dataset']
population = random.sample(range(max_uid), config['n_random'])
trial = config['n_evolution']
survivors = {}
anchestors = population.copy()
# Run Algorithm on Population
start = time.time()

print('*******************************')
print('Running Regularized Evolution algorithm')
print('Parameters:')
print(f'Dataset: {dataset}')
print(f'Num Round: {trial}')
print(f'Scoring Algorithm: {proxy_type}')
print('*******************************')
for i in range(config['n_evolution']):
  if config['proxy_type'] == 'ReLU':
    trained_population = naswot_search(dataset, device, population)
  else:
    trained_population = synflow_search(dataset, device, population)
  print('TRAINED', trained_population)
  # Take N Survivor
  trained_population.sort_values(by=['score'], ascending=False, inplace=True)
  survivors = trained_population.head(config['n_survivor'])
  print('SURVIVORS', survivors)
  # Create New Generation
  if i < trial:
    population = find_arch_n_dist(survivors=survivors, max_dist=config['n_arch_distance'], anchestors=anchestors)
  else: 
    break
  if len(population) == 0:
    break

  print(len(population), population)

stop = time.time()

total_time = stop - start
print('*****************************************************************')
print(f'Best performing net with RandomSearch')
print(tabulate(survivors, headers='keys', tablefmt='psql', showindex=False))
print(f'Total time for search over all searchspace: {total_time}')
print('*****************************************************************')
survivors.to_csv('NASWOT_REA.csv')


